# Imports

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys, os
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [2]:
dataset_name = 'ucberkeley'
model_name = 'bert-base-uncased'
run_name = "run 1"

# change to ../../results/{dataset_name} when using inside one of the data subfolders
run_folder = f'../../results/{dataset_name}/{run_name}'
test_csv_filepath = os.path.join(run_folder, 'test.csv')

model_folder = os.path.join(run_folder, model_name) # for this particular model
result_folder = os.path.join(model_folder, 'epsilon 1.0')

result_filepath = os.path.join(result_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [3]:
result = pd.read_csv(result_filepath)
test_df = pd.read_csv(test_csv_filepath)

In [4]:
test_df.head(2)

,comment_id,text,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability,labels
0,41012,I don't want to see America treated like a garbage can. Luv from Canada.,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
1,31056,@mike_pence @realDonaldTrump You're a Christian of convenience. You're a fake,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0


## Split test result

In [4]:
# only calculate test result
result = result[result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)

## Prepare test data

In [5]:
# result has id column which is the same as the text ids from raw dataset
raw_id_column = 'comment_id'
test_df.rename({raw_id_column: id_column}, axis=1, inplace=True)

## Merge

In [6]:
extra_columns = [col for col in test_df.columns if col in result.columns and col!=id_column]

result = result.merge(test_df.drop(columns=extra_columns), on=id_column, how='inner').reset_index(drop=True)
result.head(1)

,id,labels,probs,text,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability
0,41012,0,0.019339,I don't want to see America treated like a garbage can. Luv from Canada.,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Convert probability to prediction

In [7]:
result[prediction_column] = result[probability_column] >=0.5

# Evaluation

## Identity groups

In [8]:
group_map = {
    'gender': {
        'unprivileged':['target_gender_women'],
        'privileged':['target_gender_men']
    },
    'race': {
        'unprivileged':['target_race_black'],
        'privileged': ['target_race_white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['target_gender_women', 'target_gender_men', 'target_race_black', 'target_race_white']


### Binarize identity and target columns

In [9]:
result = binarize(result, [target_column] + identities)

## Bias

In [10]:
bias_results = {
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
   'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
   'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,race
0,demographic parity,0.702504,0.541825
1,Equality of Opportunity (w.r.t y = 1),0.985579,0.610042
2,Equality of Opportunity (w.r.t y = 0),0.992926,0.924775
3,Equality of Odds,0.989253,0.767409
4,unprotected-accuracy,0.729747,0.792935
5,protected-accuracy,0.762887,0.777567
6,accuracy,0.746317,0.785251


In [11]:
bias_results.round(3).to_csv(os.path.join(result_folder, 'bias.csv'), index=False)

## Overall metrics

In [12]:
overall_results = get_overall_results(group_map, result)
overall_results

,metrics,target_gender_men,target_gender_women,target_race_white,target_race_black,Total
0,auc,0.779312,0.761710,0.751867,0.850682,0.801187
1,accuracy,0.762887,0.729747,0.777567,0.792935,0.787554
2,f1_score,0.465116,0.498195,0.412060,0.740061,0.530378
3,precision,0.564516,0.634969,0.650794,0.796053,0.686449
4,recall,0.395480,0.409901,0.301471,0.691429,0.432130
5,false positive rate,0.107570,0.114644,0.056410,0.131635,0.075855
6,bnsp_auc,0.821110,0.813916,0.782288,0.886531,NaN
7,bpsn_auc,0.763736,0.766148,0.780066,0.746694,NaN


In [13]:
overall_results.round(3).to_csv(os.path.join(result_folder, 'overall_results.csv'), index=False)

## Overall bias auc

In [14]:
overall_auc = overall_results[overall_results['metrics']=='auc']['Total'].values[0]

temp = overall_results.set_index('metrics').T
privileged, unprivileged = [], []
for group_key in group_map.keys():
    privileged.extend(group_map[group_key]['privileged'])
    unprivileged.extend(group_map[group_key]['unprivileged'])

privileged_bias_auc = get_final_metric(temp[temp.index.isin(privileged)], overall_auc)
unprivileged_bias_auc = get_final_metric(temp[temp.index.isin(unprivileged)], overall_auc)

print(privileged_bias_auc, unprivileged_bias_auc)

0.7844934449723553 0.8004421018872485


# Count

The the `train.csv` file from `experiment/run` folders for this corresponding run. And manually copy it in the path run_folder points to.

In [16]:
train_df = pd.read_csv(os.path.join(run_folder, 'train.csv'))

count_dict = {
    'Identity':identities,
    '0 (train)':[],
    '1 (train)':[],
    '0 (test)':[],
    '1 (test)':[],
}
for identity in identities:
    train_neg, train_pos = train_df[train_df[identity]>=0.5][target_column].value_counts().to_numpy()
    test_neg, test_pos = test_df[test_df[identity]>=0.5][target_column].value_counts().to_numpy()
    count_dict['0 (train)'].append(train_neg)
    count_dict['1 (train)'].append(train_pos)
    count_dict['0 (test)'].append(test_neg)
    count_dict['1 (test)'].append(test_pos)

count_df = pd.DataFrame(count_dict)
print(count_df)

count_df.to_csv(os.path.join(run_folder, 'count.csv'), index=False)

              Identity  0 (train)  1 (train)  0 (test)  1 (test)
0  target_gender_women       4847       2304      1038       505
1    target_gender_men       2359        811       502       177
2    target_race_black       2136       1557       471       350
3    target_race_white       1689        634       390       136
